# Main class - `EEGReconstruction.m`

In [ ]:
%%file EEGReconstruction.m
classdef EEGReconstruction < EEGForwardModel
    % Class used to calculate the reconstruction matrices.
    %
    % EEGReconstruction Properties:
    %    SETUP - setup for simulation.
    %
    % EEGReconstruction Methods:
    %    setpreparations - adjusting signals,
    %    setfilters - calculating filters.
    
    properties(GetAccess='public', SetAccess='public')
        % Properties of the class
        RESULTS;
    end

    methods
        function obj = EEGReconstruction()
            % Constructor of EEGReconstruction class
            obj.RESULTS.table_arrC = [];
            obj.RESULTS.table_varN = {};
            obj.RESULTS.table_rowN = {};
        end
        function obj = setfilters(obj, filters)
            % Filter definitions, evaluation and calculation of
            % reconstruction errors.

            obj.MODEL = spatialfilterconstants(obj.SETUP, obj.MODEL);
            % Calculates matrices which are used in the process of reconstruction.
            for filter = filters
                obj.MODEL = spatialfiltering(obj.SETUP, obj.MODEL, filter);
            end
            obj.MODEL = spatialfilteringexecution(obj.SETUP, obj.MODEL);
            obj.MODEL = spatialfilteringerrorevaluation(obj.SETUP, obj.MODEL);
            obj.MODEL = vectorizerrorevaluation(obj.SETUP, obj.MODEL);
            
            if isempty(obj.RESULTS.table_rowN)
                np = 1;
            else
                np = length(obj.RESULTS.table_rowN) + 1;
            end
            obj.RESULTS.table_rowN{np} = obj.MODEL.rec_res.table_rowN;
            obj.RESULTS.table_varN{np} = obj.MODEL.rec_res.table_varN;
            obj.RESULTS.table_arrC(:, :, np) = obj.MODEL.rec_res.table_arrC;
        end
        function obj = printaverageresults(obj)
            % Printing average results of reconstruction for given filters.
        
            obj.RESULTS.Table = array2table(squeeze(mean(obj.RESULTS.table_arrC, 3)), 'VariableNames', table2array(cell2table(obj.RESULTS.table_varN{1})));
            obj.RESULTS.Table.Properties.RowNames = obj.RESULTS.table_rowN{1};
            disp(obj.RESULTS.Table);
        end
        function save(obj)
            % Method which saves all variables stored in the object.
            
            DATE = datestr(now, 'yyyymmdd_HHMMSS');
            save(['reconstruction_', DATE, '.mat']);
        end
    end
end

# Auxiliary functions

We assume the following notation (which in fact should be consistent with notation used in paper by T. Piotrowski et al.).
\begin{equation}
y = H q + n
\end{equation}
\begin{equation}
l := \mbox{rank}(H) = \mbox{columns}(H) \quad \color{grey}{\mbox{(this is an assumption)}}
\end{equation}
\begin{equation}
Q := \mathbb{E}[qq^{t}]
\end{equation}
\begin{equation}
N := \mathbb{E}[nn^{t}]
\end{equation}
\begin{equation}
R := H Q H^{t} + N
\end{equation}
\begin{equation}
S := H^t R^{-1} H
\end{equation}
\begin{equation}
G := H^t N^{-1} H
\end{equation}

Matrices $H$ and $H_{I}$ are given:
\begin{equation}
H := H_{Src},
\end{equation}
\begin{equation}
H_{I} := H_{Int}.
\end{equation}

However, if names here and in the paper are different, in order to avoid misunderstandings and for greater clarity we use two assignment marks $:=$. It means that the first symbol is used in the publication, and the second is used in the code below. E.g.
\begin{equation}
S_{c} := S_{SrcInt} := [H H_{I}]^{t} R^{-1} [H H_{I}],
\end{equation}
\begin{equation}
G_{c} := G_{SrcInt} := [H H_{I}]^{t} N^{-1} [H H_{I}].
\end{equation}

Let us define equivalents of the above matrices in the presence of interference
\begin{equation}
Q_{c} := \mathbb{E}[q_cq_c^{t}] = C := S^{-1} - G^{-1} = (H^t R^{-1} H)^{-1} - (H^t N^{-1} H)^{-1},
\end{equation}
\begin{equation}
Q_{c, SrcInt} := C_{SrcInt} := S_{SrcInt}^{-1} - G_{SrcInt}^{-1} = (H_{c}^t R^{-1} H_{c})^{-1} - (H_{c}^t N^{-1} H_{c})^{-1}.
\end{equation}
Moreover, let
\begin{equation}
C_{NL} := [C_{SrcInt}]_{l \times l}.
\end{equation}
Here $[A]_{k \times l}$ denotes (principal) submatrix consisting of first $k$ rows and $l$ columns of matrix $A$.

The problem of reconstruction is solved by giving a matrix called **filter**, which is the inverse of the forward solution under certain assumptions.

For example, LCMV filter $W$ is a solution to the following problem
\begin{equation}
    \begin{cases}
        \mbox{minimize} & J_R(W) := \mbox{tr}(WRW^{t}) \\
        \mbox{subject to} & W \in \mathbb{M}(l \times m), W H = I_{l}.
    \end{cases}
\end{equation}

\begin{equation}
W_{LCMV(R)} := S^{-1} H^{t} R^{-1} = (H^{t} R^{-1} H)^{-1} H^{t} R^{-1} 
\end{equation}

In [ ]:
%%file spatialfilterconstants.m
function model = spatialfilterconstants(SETUP, MODEL)
    % Calculation of covariance matrices used in filter definitions.

    model = MODEL;

    model.N = cov(reshape(permute(model.y_Pre, [1 3 2]), [], size(model.y_Pre, 2), 1));
    % model.N = cov(reshape(permute(model.y_PstNoise, [1 3 2]), [], size(model.y_PstNoise, 2), 1));
    model.R = cov(reshape(permute(model.y_Pst, [1 3 2]), [], size(model.y_Pst, 2), 1));

    model.G = model.H_Src' * pinv(model.N) * model.H_Src;
    model.S = model.H_Src' * pinv(model.R) * model.H_Src;

    model.G_SrcInt = [model.H_Src model.H_Int]' * pinv(model.N) * [model.H_Src model.H_Int];
    model.S_SrcInt = [model.H_Src model.H_Int]' * pinv(model.R) * [model.H_Src model.H_Int];
    model.C_SrcInt = pinv(model.S_SrcInt) - pinv(model.G_SrcInt);

    % Estimated C for both regular and nulling filters
    model.C_NL = model.C_SrcInt(1:size(model.H_Src, 2), 1:size(model.H_Src, 2));
    model.C = pinv(model.S) - pinv(model.G);
end

In [ ]:
%%file calculate_LCMV_R.m
function model = calculate_LCMV_R(MODEL)
    % Definition of LMCV(R) filter

    model = MODEL;
    
    model.rec_flt.LCMV_R = pinv(model.S) * model.H_Src' * pinv(model.R);
end

\begin{equation}
W_{LCMV(N)} := G^{-1} H^{t} R^{-1} = (H^{t} N^{-1} H)^{-1} H^{t} N^{-1} 
\end{equation}

In [ ]:
%%file calculate_LCMV_N.m
function model = calculate_LCMV_N(MODEL)
    % Definition of LMCV(N) filter

    model = MODEL;
    
    model.rec_flt.LCMV_N = pinv(model.G) * model.H_Src' * pinv(model.N);
end

Suppose that for $X \in \mathbb{M}(m \times n)$ we have the following singular value decomposition
\begin{equation}
X = U \Sigma V^{t},
\end{equation}
where singular values are sorted. Then $U = (u_1, \ldots, u_m)$, where column vectors $u_i$ are ortogonal. We denote
\begin{equation}
[U]_{r} := (u_1, \ldots, u_r).
\end{equation}

In what follows, we will frequently use the following fact
\begin{equation}
P_{\mathcal{R}(X)} = X X^{\dagger} = [U]_r [U]_r^t,
\end{equation}
where $r := \mbox{rank}(X)$, $X = U \Sigma V^{t}$ (see *Adi Ben-Israel and Thomas N. E. Greville. Generalized inverses: theory and applications. Vol. 15. Springer Science & Business Media, 2003*, Chapter 4: page 219, Theorem 6 (a) and page 207, Corollary 1). Moreover
\begin{equation}
P_{\mathcal{R}(X)^{\perp}} = I - P_{\mathcal{R}(X)},
\end{equation}
where $I$ is the identity matrix.

Since eigenspaces (subspaces) generated by specific eigenvectors are invariant, then
\begin{equation}
P_{X_j} := P_{\mathcal{R}(X)_j} = [U]_j [U]_j^t,
\end{equation}
where subscript $1 \leq j \leq r$ on the left-hand side means restriction of base of space to $j$ smallest eigenvectors. The reason is that for every matrix of the form $X X^{\dagger}$ (given above) vectors $u_i$, for $1 \leq i \leq r$, are its eigenvectors (easy to check). Therefore, restriction to $j$ first eigenvectors is $U \left( \begin{smallmatrix} 1 & 0 & 0 \\ 0 & \ddots & 0\\ 0 & 0 & 0 \end{smallmatrix} \right) U^{t} = [U]_j [U]_j^t$, where in the middle matrix there are only $j$ ones. If eigenvalues are sorted, then it is resection to first $j$ rows and $j$ columns.

In the derivation of the MVP filter we will use the following matrix
\begin{equation}
L^{(1)} := K_{MSE} := A^{\dagger}(A^{\dagger})^{t} - 2Q,
\end{equation}
where $A$ is given by function **calculate_P_sMVP_N_opt** below.

Then we run a loop **for j = 1:l** and calculate
\begin{equation}
J_{F}(W_{j}^{*}) = tr\left(P_{L^{(1)}_j} L^{(1)}\right) = tr\left([U^{(1)}]_{j} [U^{(1)}]_{j}^{t} L^{(1)}\right)
\end{equation}
to find $r := j$ with minimum value of the cost function $J_{F}(W_{j}^{*})$. Such $r$ is used to create and return optimal projection
\begin{equation}
P_{L^{(1)}_r} = [U^{(1)}]_{r} [U^{(1)}]_{r}^{t}.
\end{equation}
Since chosen $r$ corresponds to the smallest value of cost function, it also represents optimal number of independent sources of activity in the brain.

Functions which have **sMVP** in the name and do not have **NL** define filters that do not take into account the interference.

In [ ]:
%%file calculate_P_sMVP_MSE_opt.m
function model = calculate_P_sMVP_MSE_opt(MODEL)
    % Definition of P_sMVP(MSE) projection.

    model = MODEL;
    
    % note: pinv(model.H_Src_R)*pinv(model.H_Src_R)' = pinv(S)
    model.K_MSE = pinv(model.S) - 2*model.C;
    [model.U_K_MSE, model.W_K_MSE] = eig(model.K_MSE);
    [~, model.p_K_MSE] = sort(diag(model.W_K_MSE));
    % note: for selecting optimal rank, Theobald's Theorem from
    % M. Theobald. *An inequality for the trace of the product of
    % two symmetric matrices*. Volume 77, Issue 2 March 1975, pp. 265-267
    % can be used without evaluation of the cost function, or it can be
    % done as here:
    model.U_K_MSE = model.U_K_MSE(:, model.p_K_MSE);
    model.sMVP_MSE_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        model.sMVP_MSE_ranks(jj) = trace(model.U_K_MSE(:, 1:jj) * model.U_K_MSE(:, 1:jj)' * model.K_MSE);
    end

    [~, model.sMVP_MSE_rank_opt] = min(model.sMVP_MSE_ranks);
    % _opt stands for optimal
    model.P_sMVP_MSE_opt = model.U_K_MSE(:, 1:model.sMVP_MSE_rank_opt) * model.U_K_MSE(:, 1:model.sMVP_MSE_rank_opt)';
end

Let us observe that $S^{-1} = A^{\dagger}(A^{\dagger})^{t}$ (just use SVD decomposition of $A$), so we can write
\begin{equation}
L^{(1)} := K_{MSE} := S^{-1} - 2Q.
\end{equation}
Let us define
\begin{equation}
L^{(2)}:= K_{R} := L^{(1)} + 2 Q = S^{-1} = A^{\dagger}(A^{\dagger})^{t}.
\end{equation}
Similarly as above run a loop **for j = 1:l** and calculate
\begin{equation}
J_{F}(W_{j}^{*}) = tr\left(P_{L^{(2)}_j} L^{(1)}\right) = tr\left([U^{(2)}]_{j} [U^{(2)}]_{j}^{t} L^{(2)}\right)
\end{equation}
to find $r := j$ with minimum value of the cost function $J_{F}(W_{j}^{*})$. Such $r$ is used to create and return optimal projection by
\begin{equation}
P_{L^{(1)}_r} := [U^{(2)}]_{r} [U^{(2)}]_{r}^{t}.
\end{equation}

In [ ]:
%%file calculate_P_sMVP_R_opt.m
function model = calculate_P_sMVP_R_opt(MODEL)
    % Definition of P_sMVP(R) projection.

    model = MODEL;
    
    % note: pinv(model.H_Src_R)*pinv(model.H_Src_R)' = pinv(S)
    model.K_MSE = pinv(model.S) - 2*model.C;

    model.K_R = model.K_MSE + 2*model.C;
    [model.U_K_R, model.W_K_R] = eig(model.K_R);
    [~, model.p_K_R] = sort(diag(model.W_K_R));
    model.U_K_R = model.U_K_R(:, model.p_K_R);
    model.sMVP_R_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        model.sMVP_R_ranks(jj) = trace(model.U_K_R(:, 1:jj) * model.U_K_R(:, 1:jj)' * model.K_MSE);
    end

    [~, model.sMVP_R_rank_opt] = min(model.sMVP_R_ranks);
    % _opt stands for optimal
    model.P_sMVP_R_opt = model.U_K_R(:, 1:model.sMVP_R_rank_opt) * model.U_K_R(:, 1:model.sMVP_R_rank_opt)';
end

Let us define
\begin{equation}
L^{(4)} := K_{MSE(N)} := B^{\dagger}(B^{\dagger})^{t} - Q = G^{-1} - Q,
\end{equation}
\begin{equation}
L^{(3)} := K^{N}:= K_{MSE(N)} + Q = G^{-1} = B^{\dagger}(B^{\dagger})^{t},
\end{equation}
where $B$ is given by function **calculate_P_sMVP_N_opt** below.
Similarly as above we run a loop **for j = 1:l** and calculate
\begin{equation}
J_{F}(W_{j}^{*}) = tr\left(P_{L^{(3)}_j} L^{(4)}\right) = tr\left([U^{(3)}]_{j} [U^{(3)}]_{j}^{t} L^{(4)}\right)
\end{equation}
to find $r := j$ with minimum value of the cost function $J_{F}(W_{j}^{*})$. Such $r$ is used to create and return optimal projection by
\begin{equation}
P_{L^{(3)}_r} := [U^{(3)}]_{r} [U^{(3)}]_{r}^{t}.
\end{equation}

In [ ]:
%%file calculate_P_sMVP_N_opt.m
function model = calculate_P_sMVP_N_opt(MODEL)
    % Definition of P_sMVP(N) projection.

    model = MODEL;
    
    model.K_MSE_N = pinv(model.G) - model.C;

    model.K_N = model.K_MSE_N + model.C;
    [model.U_K_N, model.W_K_N] = eig(model.K_N);
    [~, model.p_K_N] = sort(diag(model.W_K_N));
    model.U_K_N = model.U_K_N(:, model.p_K_N);
    model.sMVP_N_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        model.sMVP_N_ranks(jj) = trace(model.U_K_N(:, 1:jj) * model.U_K_N(:, 1:jj)' * model.K_MSE_N);
    end

    [~, model.sMVP_N_rank_opt] = min(model.sMVP_N_ranks);
    % _opt stands for optimal
    model.P_sMVP_N_opt = model.U_K_N(:, 1:model.sMVP_N_rank_opt) * model.U_K_N(:, 1:model.sMVP_N_rank_opt)';
end

\begin{equation}
A := H_{Src, R} := R^{-1/2} H
\end{equation}
\begin{equation}
B := H_{Src, N} := N^{-1/2} H
\end{equation}

In [ ]:
%%file calculate_H_Src.m
function model = calculate_H_Src(MODEL)
    % Auxiliary matrices used in filter definitions.

    model = MODEL;
    
    model.H_Src_R = pinv(sqrtm(model.R)) * model.H_Src;
    model.H_Src_N = pinv(sqrtm(model.N)) * model.H_Src;
end

\begin{equation}
A_{I} := H_{Int, R} := R^{-1/2} H_I
\end{equation}
\begin{equation}
B_{I} := H_{Int, N} := N^{-1/2} H_I
\end{equation}

In [ ]:
%%file calculate_H_Int.m
function model = calculate_H_Int(MODEL)
    % Auxiliary matrices used in filter definitions.

    model = MODEL;
    
    model.H_Int_R = pinv(sqrtm(model.R)) * model.H_Int;
    model.H_Int_N = pinv(sqrtm(model.N)) * model.H_Int;
end

\begin{equation}
P_{\mathcal{R}(A_{I})^{\perp}} := P_{sMVP_{NL_R}} := I - A_{I} A_{I}^{\dagger}
\end{equation}
\begin{equation}
P_{\mathcal{R}(B_{I})^{\perp}} := P_{sMVP_{NL_N}} := I - B_{I} B_{I}^{\dagger}
\end{equation}

In [ ]:
%%file calculate_sMVP_NL.m
function model = calculate_sMVP_NL(MODEL)
    % Auxiliary matrices used in filter definitions.

    model = MODEL;
    
    model.P_sMVP_NL_R = eye(size(model.H_Int_R, 1)) - model.H_Int_R * pinv(model.H_Int_R);
    model.P_sMVP_NL_N = eye(size(model.H_Int_N, 1)) - model.H_Int_N * pinv(model.H_Int_N);
end

Let's define a set that will be a set, over which we will optimize our cost functions
\begin{equation}
\mathcal{X}_{r}^{l \times m} := \{ A \in \mathbb{M}(l \times m) \mid \mbox{rank}(A) \leq r \leq \min(l, k) \}.
\end{equation}
What is importat, $\mathcal{X}_{r}^{l \times m}$ is not convex, therefore typical optimization algorithms do not work.

In what follows an important role will play by the following set
\begin{equation}
\mathcal{P}_{r}^{\gamma} := \mbox{arg} \min_{W_{r} \in \mathcal{X}_{r}^{l \times m}} \|W_{r} H - I_{l}\|^2_{\gamma}, \quad \gamma \in \Gamma.
\end{equation}

Let us define cost function in the presence of interference as
\begin{equation}
J_{I}(W) := \mbox{tr} \big( \mathbb{E}[(W(H_c q_c + n) - q)(W(H_c q_c + n) - q)^{t} ]\big).
\end{equation}

When considering reconstruction with intereference, we have the following problem
\begin{equation}
    \begin{cases}
        \mbox{minimize} & J_R(W) := \mbox{tr}(WRW^{t}) \\
        \mbox{subject to} & W \in \mathbb{M}(l \times m), W H = I_{l}, W H_{I} = 0_{l \times k},
    \end{cases}
\end{equation}
which basically means that sources which interfere with signals associated with brain activity are lineary dependent.  (without the last constrain the solution is [BLUE](https://en.wikipedia.org/wiki/Gauss%E2%80%93Markov_theorem), i.e. LCMV filter) Furthermore, we assume that this dependency is contained in matrix $W$. The solution to the above-mentioned problem is
\begin{equation}
W_{NL} := [I_l 0_{l \times k}] (H_{c}^{t} R^{-1} H_{c})^{-1} H_{c}^{t} R^{-1}.
\end{equation}

**Theorem 1**: Closed algebraic form for Reduced-rank Nulling filter $W$ satisfying
\begin{equation}
    \begin{cases}
        \mbox{minimize} & J_I(W), \\
        \mbox{subject to} & W \in \bigcap_{\gamma \in \Gamma} \mathcal{P}_{r}^{\gamma}, W H_{I} = 0_{l \times k},
    \end{cases}
\end{equation}
is
\begin{equation}
W_{NL(MSE)}:= W_r^* = P_{K^{(1)}_r} \left( P_{\mathcal{R}(A_{I})^{\perp}} A \right)^{\dagger} P_{\mathcal{R}(A_{I})^{\perp}} R^{-1/2},
\end{equation}
where
\begin{equation}
K^{(1)} := \left( P_{\mathcal{R}(A_{I})^{\perp}} A \right)^{\dagger} P_{\mathcal{R}(A_{I})^{\perp}} \left( (P_{\mathcal{R}(A_{I})^{\perp}} A)^{\dagger} \right)^{t} - 2Q
\end{equation}
and the cost function can be expressed as
\begin{equation}
J_{I}(W_{r}^{*}) = tr(P_{K^{(1)}_r} K^{(1)}) + c.
\end{equation}
Term *nulling* (**sMVP\_NL\_*** in code) herein means that the filter takes into account the interference.

In [ ]:
%%file calculate_sMVP_NL_MSE.m
function model = calculate_sMVP_NL_MSE(MODEL)
    % Definition of sMVP_NL_MSE filter.

    model = MODEL;
    
    model.K_NL_MSE = pinv(model.P_sMVP_NL_R * model.H_Src_R) * model.P_sMVP_NL_R * (pinv(model.P_sMVP_NL_R * model.H_Src_R))' - 2*model.C_NL;
    [model.U_K_NL_MSE, model.W_K_NL_MSE] = eig(model.K_NL_MSE);
    [~, model.p_K_NL_MSE] = sort(diag(model.W_K_NL_MSE));
    % note: for selecting optimal rank, Theobald's Theorem can be
    % used without evaluation of the cost function, or it can be
    % done as here:
    model.U_K_NL_MSE = model.U_K_NL_MSE(:, model.p_K_NL_MSE);
    model.sMVP_NL_MSE_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        % cost function
        model.sMVP_NL_MSE_ranks(jj) = trace(model.U_K_NL_MSE(:, 1:jj) * model.U_K_NL_MSE(:, 1:jj)' * model.K_NL_MSE);
    end

    [~, model.sMVP_NL_MSE_rank_opt] = min(model.sMVP_NL_MSE_ranks);
    % _opt stands for optimal
    model.P_sMVP_NL_MSE_opt = model.U_K_NL_MSE(:, 1:model.sMVP_NL_MSE_rank_opt) * model.U_K_NL_MSE(:, 1:model.sMVP_NL_MSE_rank_opt)';

    model.rec_opt.ranks.sMVP_NL_MSE = model.sMVP_NL_MSE_rank_opt;

    model.rec_flt.sMVP_NL_MSE = model.P_sMVP_NL_MSE_opt * pinv(model.P_sMVP_NL_R * model.H_Src_R) * model.P_sMVP_NL_R * pinv(sqrtm(model.R));
end

**Theorem 2**: Closed algebraic form for Reduced-rank Nulling filter $W$ satisfying
\begin{equation}
    \begin{cases}
        \mbox{minimize} & J_{R}(W) := \mbox{tr} (W R W^{t}), \\
        \mbox{subject to} & W \in \bigcap_{\gamma \in \Gamma} \mathcal{P}_{r}^{\gamma}, W H_{I} = 0_{l \times k},
    \end{cases}
\end{equation}
i.e. using matrix $R$, is
\begin{equation}
W_r^* = P_{K^{(2)}_r} \left( P_{\mathcal{R}(A_{I})^{\perp}} A \right)^{\dagger} P_{\mathcal{R}(A_{I})^{\perp}} R^{-1/2},
\end{equation}
where
\begin{equation}
K^{(2)} := K^{(1)} + 2Q = \left( P_{\mathcal{R}(A_{I})^{\perp}} A \right)^{\dagger} P_{\mathcal{R}(A_{I})^{\perp}} \left( (P_{\mathcal{R}(A_{I})^{\perp}} A)^{\dagger} \right)^{t},
\end{equation}
and the cost function can be expressed as
\begin{equation}
J_R(W_{r}^{*}) = tr(P_{K^{(2)}_r} K^{(1)}) + c.
\end{equation}

Moreover,
\begin{equation}
W_{NL(R)} = \left( P_{\mathcal{R}(A_{I})^{\perp}} A \right)^{\dagger} P_{\mathcal{R}(A_{I})^{\perp}} R^{-1/2}.
\end{equation}

In [ ]:
%%file calculate_sMVP_NL_R.m
function model = calculate_sMVP_NL_R(MODEL)
    % Definition of sMVP_NL_R filter.

    model = MODEL;
    
    model.K_NL_MSE = pinv(model.P_sMVP_NL_R * model.H_Src_R) * model.P_sMVP_NL_R * (pinv(model.P_sMVP_NL_R * model.H_Src_R))' - 2*model.C_NL;

    %K_NL_R = pinv(model.P_sMVP_NL_R * model.H_Src_R) * model.P_sMVP_NL_R * (pinv(model.P_sMVP_NL_R * model.H_Src_R))';
    model.K_NL_R = model.K_NL_MSE + 2*model.C_NL;
    [model.U_K_NL_R, model.W_K_NL_R] = eig(model.K_NL_R);

    [~, model.p_K_NL_R] = sort(diag(model.W_K_NL_R));
    model.U_K_NL_R = model.U_K_NL_R(:, model.p_K_NL_R);
    model.sMVP_NL_R_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        % cost function
        model.sMVP_NL_R_ranks(jj) = trace(model.U_K_NL_R(:, 1:jj) * model.U_K_NL_R(:, 1:jj)' * model.K_NL_MSE);
    end

    [~, model.sMVP_NL_R_rank_opt] = min(model.sMVP_NL_R_ranks);
    % _opt stands for optimal
    model.P_sMVP_NL_R_opt = model.U_K_NL_R(:, 1:model.sMVP_NL_R_rank_opt) * model.U_K_NL_R(:, 1:model.sMVP_NL_R_rank_opt)';

    model.rec_opt.ranks.sMVP_NL_R = model.sMVP_NL_R_rank_opt;

    model.rec_flt.sMVP_NL_R = model.P_sMVP_NL_R_opt * pinv(model.P_sMVP_NL_R * model.H_Src_R) * model.P_sMVP_NL_R * pinv(sqrtm(model.R));
end

**Theorem 3**: Closed algebraic form for Reduced-rank Nulling filter $W$ satisfying
\begin{equation}
    \begin{cases}
        \mbox{minimize} & J_{N}(W) := \mbox{tr} (W N W^{t}), \\
        \mbox{subject to} & W \in \bigcap_{\gamma \in \Gamma} \mathcal{P}_{r}^{\gamma}, W H_{I} = 0_{l \times k},
    \end{cases}
\end{equation}
i.e. using matrix $N$, is
\begin{equation}
W_{r}^{*} = P_{K^{(3)}_r} \left( P_{\mathcal{R}(B_{I})^{\perp}} B \right)^{\dagger} P_{\mathcal{R}(B_{I})^{\perp}} N^{-1/2},
\end{equation}
where
\begin{equation}
K^{(3)} := \left( P_{\mathcal{R}(B_{I})^{\perp}} B \right)^{\dagger} P_{\mathcal{R}(B_{I})^{\perp}} \left( (P_{\mathcal{R}(B_{I})^{\perp}} B)^{\dagger} \right)^{t},
\end{equation}
\begin{equation}
K^{(4)} := K^{(3)} - Q,
\end{equation}
and the cost function can be expressed as
\begin{equation}
J_N(W_{r}^{*}) = tr(P_{K^{(3)}_r} K^{(4)}) + c.
\end{equation}

Actually
\begin{equation}
J_N(W) = J_F(W) = J_R(W) - c,
\end{equation}
for any $W$ satisfying $WH = I_{l}$.

Moreover,
\begin{equation}
W_{NL(N)} = \left( P_{\mathcal{R}(B_{I})^{\perp}} B \right)^{\dagger} P_{\mathcal{R}(B_{I})^{\perp}} N^{-1/2}.
\end{equation}

In [ ]:
%%file calculate_sMVP_NL_N.m
function model = calculate_sMVP_NL_N(MODEL)
    % Definition of sMVP_NL_N filter.

    model = MODEL;
    
    model.K_NL_MSE_N = pinv(model.P_sMVP_NL_N * model.H_Src_N) * model.P_sMVP_NL_N * (pinv(model.P_sMVP_NL_N * model.H_Src_N))' - model.C_NL;

    %K_NL_N = pinv(model.P_sMVP_NL_N * model.H_Src_N) * model.P_sMVP_NL_N * (pinv(model.P_sMVP_NL_N * model.H_Src_N))';
    model.K_NL_N = model.K_NL_MSE_N + model.C_NL;
    [model.U_K_NL_N, model.W_K_NL_N] = eig(model.K_NL_N);
    [~, model.p_K_NL_N] = sort(diag(model.W_K_NL_N));
    model.U_K_NL_N = model.U_K_NL_N(:, model.p_K_NL_N);
    model.sMVP_NL_N_ranks = zeros(1, size(model.H_Src, 2));

    for jj = 1:size(model.H_Src, 2)
        % cost function
        model.sMVP_NL_N_ranks(jj) = trace(model.U_K_NL_N(:, 1:jj) * model.U_K_NL_N(:, 1:jj)' * model.K_NL_MSE_N);
    end

    [~, model.sMVP_NL_N_rank_opt] = min(model.sMVP_NL_N_ranks);
    % note the simplified notation for P_N_opt
    model.P_sMVP_NL_N_opt = model.U_K_NL_N(:, 1:model.sMVP_NL_N_rank_opt) * model.U_K_NL_N(:, 1:model.sMVP_NL_N_rank_opt)';

    model.rec_opt.ranks.sMVP_NL_N = model.sMVP_NL_N_rank_opt;

    model.rec_flt.sMVP_NL_N = model.P_sMVP_NL_N_opt * pinv(model.P_sMVP_NL_N * model.H_Src_N) * model.P_sMVP_NL_N * pinv(sqrtm(model.N));
end

Below is a list of filters implemented in the library:
\begin{equation}
W_{MMSE} := Q H^{t} R^{-1} \quad \color{grey}{\mbox{Wiener filter}}
\end{equation}
\begin{equation}
W_{ZF} := H^{\dagger}
\end{equation}
\begin{equation}
W_{EIG-LCMV(R)} := W_{LCMV(R)} P_{R_{sig}}
\end{equation}
\begin{equation}
W_{EIG-LCMV(N)} := W_{LCMV(N)} P_{R_{sig}}
\end{equation}
Filters $W_{NL(MSE)}$, $W_{NL(R)}$, $W_{NL(N)}$ (**sMVP\_NL\_*** in code) were calculated in Theorems 1-3 above.
\begin{equation}
W_{NL} := [I_l 0_{l \times k}] (H_{c}^{t} R^{-1} H_{c})^{-1} H_{c}^{t} R^{-1}
\end{equation}

In theory $W_{NL} = W_{NL(R)} =  W_{NL(N)}$. However, in practice (with specific estimators for $R$ and $N$) they can be different.

In [ ]:
%%file spatialfiltering.m
function model = spatialfiltering(SETUP, MODEL, filter)
    % Definition of all filters.

    model = MODEL;

    switch filter
    
        case "LCMV"
            % note: S and G use only model.H_Src, as we want to recover
            % only activity of interest, not interference 
            model = calculate_LCMV_R(model);
            model = calculate_LCMV_N(model);
        
        case "MMSE"
            %  MMSE filter
            model.rec_flt.MMSE = model.C * model.H_Src' * pinv(model.R);
            
        case "ZF"
            model.rec_flt.ZF = pinv(model.H_Src);
            
        case "RANDN"
            model.rec_flt.RANDN = randn(size(model.H_Src'));
            
        case "ZEROS"
            model.rec_flt.ZEROS = zeros(size(model.H_Src'));

        case "EIG-LCMV"
            %  EIG-LCMV filter
            [model.UR, model.SiR, ~] = svd(model.R);
            model.P_EIG = model.UR(:, 1:SETUP.RANK_EIG) * model.UR(:, 1:SETUP.RANK_EIG)';

            model = calculate_LCMV_R(model);
            model = calculate_LCMV_N(model);
            
            model.rec_flt.EIG_LCMV_R = model.rec_flt.LCMV_R * model.P_EIG;
            model.rec_flt.EIG_LCMV_N = model.rec_flt.LCMV_N * model.P_EIG;

        case "sMVP_MSE"
            % sMVP_MSE filter
            % note: S and G use only model.H_Src, as we want to recover only
            % activity of interest, not interference
            % note: pinv(model.H_Src_R)*pinv(model.H_Src_R)' = pinv(S)
            model = calculate_LCMV_R(model);
            model = calculate_P_sMVP_MSE_opt(model);
            model.rec_opt.ranks.sMVP_MSE = model.sMVP_MSE_rank_opt;
            model.rec_flt.sMVP_MSE = model.P_sMVP_MSE_opt * model.rec_flt.LCMV_R;

        case "sMVP_R"
            % sMVP_R filter
            % note: S and G use only model.H_Src, as we want to recover only
            % activity of interest, not interference
            % note: pinv(model.H_Src_R)*pinv(model.H_Src_R)' = pinv(S)
            model = calculate_LCMV_R(model);
            model = calculate_P_sMVP_R_opt(model);
            model.rec_opt.ranks.sMVP_R = model.sMVP_R_rank_opt;
            model.rec_flt.sMVP_R = model.P_sMVP_R_opt * model.rec_flt.LCMV_R;

        case "sMVP_N"
            % sMVP_N filter
            % note: S and G use only model.H_Src, as we want to recover only
            % activity of interest, not interference
            % note: pinv(model.H_Src_N)*pinv(model.H_Src_N)' = pinv(G)
            model = calculate_LCMV_N(model);
            model = calculate_P_sMVP_N_opt(model);
            model.rec_opt.ranks.sMVP_N = model.sMVP_N_rank_opt;
            model.rec_flt.sMVP_N = model.P_sMVP_N_opt * model.rec_flt.LCMV_N;

        case "sMVP_NL_MSE"
            % sMVP-NL filters
            model = calculate_H_Src(model);
            model = calculate_H_Int(model);
            model = calculate_sMVP_NL(model);

            % sMVP_NL_MSE filter
            model = calculate_sMVP_NL_MSE(model);

        case "sMVP_NL_R"
            % sMVP-NL filters
            model = calculate_H_Src(model);
            model = calculate_H_Int(model);
            model = calculate_sMVP_NL(model);
        
            % sMVP_NL_R filter
            model = calculate_sMVP_NL_R(model);

        case "sMVP_NL_N"
            % sMVP-NL filters
            model = calculate_H_Src(model);
            model = calculate_H_Int(model);
            model = calculate_sMVP_NL(model);
            
            % sMVP_NL_N filter
            model = calculate_sMVP_NL_N(model);

        otherwise
            % NL filter as proposed in Hui & Leahy 2010
            model = calculate_H_Src(model);
            model = calculate_H_Int(model);

            model.H_SrcInt = [model.H_Src model.H_Int];
            P_NL = [eye(size(model.H_Src, 2)) zeros(size(model.H_Src, 2), size(model.H_Int, 2))];
            
            model.rec_flt.NL = P_NL * pinv(model.H_SrcInt' * pinv(model.R) * model.H_SrcInt) * model.H_SrcInt' * pinv(model.R);
    end
    
    if(SETUP.fltREMOVE) && (isfield(model.rec_flt, 'ZF'))
        model.rec_flt = rmfield(model.rec_flt, 'ZF');
    end
    if(SETUP.fltREMOVE) && (isfield(model.rec_flt, 'RANDN'))
        model.rec_flt = rmfield(model.rec_flt, 'RANDN');
    end
    if(SETUP.fltREMOVE) && (isfield(model.rec_flt, 'ZEROS'))
        model.rec_flt = rmfield(model.rec_flt, 'ZEROS');
    end

end

In [ ]:
%%file spatialfilteringexecution.m
function model = spatialfilteringexecution(SETUP, MODEL)
    % Reconstruction of source signals using spatial filter and
    % fitting MVAR to the reconstructed signal.

    model = MODEL;
    
    model.rec_sig.Original = model.sim_sig_SrcActiv.sigSRC_pst;
    model.rec_sig.Dummy = model.sim_sig_SrcActiv.sigSRC_pre;
    tmp_fltFields = fieldnames(model.rec_flt);

    for nn = 1:length(tmp_fltFields) % For all filters
        for kk = 1:SETUP.K00 % For each realisation
            model.rec_sig.(tmp_fltFields{nn})(:, :, kk) = (model.rec_flt.(tmp_fltFields{nn}) * model.y_Pst(:, :, kk)')'; % We reconstruct using on y_Pst
        end
    end
    
    model.rec_funDep_A00.Original = model.sim_sig_SrcActiv.A00;

    [~, model.rec_funDep_A00.Dummy, ~, ~, ~, ~] = arfit(model.sim_sig_SrcActiv.sigSRC_pre, model.sim_sig_SrcActiv.P00, model.sim_sig_SrcActiv.P00);

    for nn = 1:length(tmp_fltFields)
        [~, model.rec_funDep_A00.(tmp_fltFields{nn}), ~, ~, ~, ~] = arfit(model.rec_sig.(tmp_fltFields{nn}), SETUP.P00,SETUP.P00);
    end

    tmp_allFields = fieldnames(model.rec_funDep_A00);
    for nn = 1:length(tmp_allFields)
        [mDTF, mPDC] = EEGStaticMethods().myPDC(model.rec_funDep_A00.(tmp_allFields{nn}), SETUP.PDC_RES);
        model.rec_funDep_PDC.(tmp_allFields{nn}) = abs(mPDC);
        model.rec_funDep_DTF.(tmp_allFields{nn}) = abs(mDTF);
    end
end

In [ ]:
%%file spatialfilteringerrorevaluation.m
function model = spatialfilteringerrorevaluation(SETUP, MODEL)
    % Error evaluation using Euclidan and Correlation Coefficients.

    model = MODEL;
    
    tmp_allFields  = fieldnames(model.rec_sig);

    % Preparation
    for nn = 1:length(tmp_allFields)
      for ii = 1:SETUP.n00
          for jj = 1:SETUP.K00
              model.rec_sig.(tmp_allFields{nn})(ii, :, jj) = model.rec_sig.(tmp_allFields{nn})(ii, :, jj) / norm(model.rec_sig.(tmp_allFields{nn})(ii, :, jj));
          end
      end
    end
    
    % Euclid for signal
    for nn = 1:length(tmp_allFields)
      tmp_error = zeros(SETUP.n00, SETUP.K00);
      for ii = 1:SETUP.n00
          for jj = 1:SETUP.K00
              tmp_error(ii, jj) = norm(model.rec_sig.(tmp_allFields{1})(ii, :, jj) - model.rec_sig.(tmp_allFields{nn})(ii, :, jj))^2;                  
          end
      end
      model.rec_sigAmp_ErrEuclid.(tmp_allFields{nn}) = mean(mean(tmp_error));
    end

    % CorrCf for signal
    for nn = 1:length(tmp_allFields)
      tmp_error = zeros(sum(SETUP.SRCS(:, 1)), SETUP.K00);
      for kk = 1:sum(SETUP.SRCS(:, 1))
          for jj = 1:SETUP.K00
              tmp_x = corrcoef(model.rec_sig.(tmp_allFields{1})(:, kk, jj), model.rec_sig.(tmp_allFields{nn})(:, kk, jj));
              tmp_error(kk, jj) = tmp_x(1, end);
          end
      end
      model.rec_sigAmp_ErrCorrCf.(tmp_allFields{nn}) = mean(mean(tmp_error));
    end

    % Euclid for MVAR coefficients matrix
    for nn = 1:length(tmp_allFields)
      model.rec_funDep_A00_ErrEuclid.(tmp_allFields{nn}) = (norm(model.rec_funDep_A00.(tmp_allFields{1}) - model.rec_funDep_A00.(tmp_allFields{nn}), 'fro')^2);
    end

    % CorrCf for MVAR coefficient matrix
    for nn = 1:length(tmp_allFields)
      tmp_error = zeros(sum(SETUP.SRCS(:, 1)), 1);
      for kk = 1:sum(SETUP.SRCS(:, 1))
          tmp_x = corrcoef(model.rec_funDep_A00.(tmp_allFields{1})(kk, :), model.rec_funDep_A00.(tmp_allFields{nn})(kk, :));
          tmp_error(kk) = tmp_x(1, end);
      end
      model.rec_funDep_A00_ErrCorrCf.(tmp_allFields{nn}) = mean(tmp_error);
    end

    % Euclid for PDC coefficient matrix
    for nn = 1:length(tmp_allFields)
      tmp_error = zeros(sum(SETUP.SRCS(:,1)),1);
      for ii = 1:sum(SETUP.SRCS(:, 1))
          tmp_error(ii) = norm(squeeze(model.rec_funDep_PDC.(tmp_allFields{1})(ii, :, :)) - squeeze(model.rec_funDep_PDC.(tmp_allFields{nn})(ii, :, :)), 'fro')^2;
      end
      model.rec_funDep_PDC_ErrEuclid.(tmp_allFields{nn}) = mean(tmp_error);
    end

    % CorrCf for PDC coefficient matrix
    for nn = 1:length(tmp_allFields)
      tmp_error = zeros(sum(SETUP.SRCS(:, 1)), 1);
      for ii = 1:sum(SETUP.SRCS(:, 1))
          tmp_x = corrcoef(reshape(squeeze(model.rec_funDep_PDC.(tmp_allFields{1})(ii, :, :)), [], 1), reshape(squeeze(model.rec_funDep_PDC.(tmp_allFields{nn})(ii, :, :)), [], 1));
          tmp_error(ii) = tmp_x(1, end);
      end
      model.rec_funDep_PDC_ErrCorrCf.(tmp_allFields{nn}) = mean(tmp_error);
    end
end

In [ ]:
%%file vectorizerrorevaluation.m
function model = vectorizerrorevaluation(SETUP, MODEL)
    % Auxiliary function for plotting error evaluations.

    model = MODEL;

    % Vectorize results
    model.rec_sigAmp_ErrEuclid_vec = cellfun(@(fn) model.rec_sigAmp_ErrEuclid.(fn), fieldnames(model.rec_sigAmp_ErrEuclid), 'UniformOutput', false);
    model.rec_sigAmp_ErrEuclid_vec = squeeze(vertcat(model.rec_sigAmp_ErrEuclid_vec{:}));

    model.rec_sigAmp_ErrCorrCf_vec = cellfun(@(fn) model.rec_sigAmp_ErrCorrCf.(fn), fieldnames(model.rec_sigAmp_ErrCorrCf), 'UniformOutput', false);
    model.rec_sigAmp_ErrCorrCf_vec = squeeze(vertcat(model.rec_sigAmp_ErrCorrCf_vec{:}));

    model.rec_funDep_A00_ErrEuclid_vec = cellfun(@(fn) model.rec_funDep_A00_ErrEuclid.(fn), fieldnames(model.rec_funDep_A00_ErrEuclid), 'UniformOutput', false);
    model.rec_funDep_A00_ErrEuclid_vec = squeeze(vertcat(model.rec_funDep_A00_ErrEuclid_vec{:}));

    model.rec_funDep_A00_ErrCorrCf_vec = cellfun(@(fn) model.rec_funDep_A00_ErrCorrCf.(fn), fieldnames(model.rec_funDep_A00_ErrCorrCf), 'UniformOutput', false);
    model.rec_funDep_A00_ErrCorrCf_vec = squeeze(vertcat(model.rec_funDep_A00_ErrCorrCf_vec{:}));

    model.rec_funDep_PDC_ErrEuclid_vec = cellfun(@(fn) model.rec_funDep_PDC_ErrEuclid.(fn), fieldnames(model.rec_funDep_PDC_ErrEuclid), 'UniformOutput', false);
    model.rec_funDep_PDC_ErrEuclid_vec = squeeze(vertcat(model.rec_funDep_PDC_ErrEuclid_vec{:}));

    model.rec_funDep_PDC_ErrCorrCf_vec = cellfun(@(fn) model.rec_funDep_PDC_ErrCorrCf.(fn), fieldnames(model.rec_funDep_PDC_ErrCorrCf), 'UniformOutput', false);
    model.rec_funDep_PDC_ErrCorrCf_vec = squeeze(vertcat(model.rec_funDep_PDC_ErrCorrCf_vec{:}));

    % Combine results in single array
    if     ~isequal(fieldnames(model.rec_sigAmp_ErrEuclid), fieldnames(model.rec_sigAmp_ErrCorrCf))
        error('check fieldnames');
    elseif ~isequal(fieldnames(model.rec_sigAmp_ErrEuclid), fieldnames(model.rec_funDep_A00_ErrEuclid))
        error('check fieldnames');
    elseif ~isequal(fieldnames(model.rec_sigAmp_ErrEuclid), fieldnames(model.rec_funDep_A00_ErrCorrCf))
        error('check fieldnames');
    elseif ~isequal(fieldnames(model.rec_sigAmp_ErrEuclid), fieldnames(model.rec_funDep_PDC_ErrEuclid))
        error('check fieldnames');
    elseif ~isequal(fieldnames(model.rec_sigAmp_ErrEuclid), fieldnames(model.rec_funDep_PDC_ErrCorrCf))
        error('check fieldnames');
    else
        model.rec_res.table_arrC = [ model.rec_sigAmp_ErrEuclid_vec,  model.rec_sigAmp_ErrCorrCf_vec,  model.rec_funDep_A00_ErrEuclid_vec,  model.rec_funDep_A00_ErrCorrCf_vec,  model.rec_funDep_PDC_ErrEuclid_vec,  model.rec_funDep_PDC_ErrCorrCf_vec];
        model.rec_res.table_varN = {'rec_sigAmp_ErrEuclid_vec', 'rec_sigAmp_ErrCorrCf_vec', 'rec_funDep_A00_ErrEuclid_vec', 'rec_funDep_A00_ErrCorrCf_vec', 'rec_funDep_PDC_ErrEuclid_vec', 'rec_funDep_PDC_ErrCorrCf_vec'};
        model.rec_res.table_rowN = fieldnames(model.rec_sigAmp_ErrEuclid);
        model.rec_res.table = array2table(model.rec_res.table_arrC, 'VariableNames', model.rec_res.table_varN, 'RowNames', model.rec_res.table_rowN);
    end

    if SETUP.DEBUG
        model.rec_res.table_arrC = [ model.rec_sigAmp_ErrEuclid_vec,  model.rec_sigAmp_ErrCorrCf_vec ];
        model.rec_res.table_varN = {'rec_sigAmp_ErrEuclid_vec', 'rec_sigAmp_ErrCorrCf_vec'};
        model.rec_res.table_rowN = fieldnames(model.rec_sigAmp_ErrEuclid);
        model.rec_res.table = array2table(model.rec_res.table_arrC, 'VariableNames', model.rec_res.table_varN, 'RowNames', model.rec_res.table_rowN);
    end
end